In [ ]:
#1 Gerekli kütüphaneleri içe aktarma
# - TensorFlow ve Keras: Model oluşturma ve eğitim için
# - ImageDataGenerator: Görüntü ön işleme ve artırımı
# - Matplotlib: Grafikler için
# - NumPy: Sayısal işlemler
# - Scikit-learn: Sınıf ağırlıkları ve metrikler
# - PIL: Görüntü doğrulama
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.utils import class_weight
from PIL import Image

In [ ]:
#2 Parametreleri tanımlama
# - img_size: Görüntü boyutu (bellek tasarrufu için küçük)
# - batch_size: Düşük batch boyutu, bellek yükünü azaltır
# - data_dir: Veri seti klasörü (uav/, other/)
# - epochs: Erken durdurma ile optimize edilmiş
img_size = (64, 64)  # Bellek tasarrufu için küçültüldü
batch_size = 16  # Çökme riskini azaltmak için
data_dir = 'cnndata/'  # cnndata/uav/, cnndata/other/
epochs = 3  # Hızlı test için düşük epoch sayısı

In [ ]:
#3 Veri setini doğrulama
# - Bozuk görüntüleri tespit eder
def check_images(directory):
    invalid_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()
            except Exception as e:
                invalid_files.append(f"{file}: {e}")
    if invalid_files:
        print("Bozuk dosyalar:", invalid_files)
    else:
        print("Tüm görüntüler geçerli.")
    return len(invalid_files) == 0

if not check_images(data_dir):
    raise ValueError("Bozuk görüntüler tespit edildi, lütfen veri setini düzeltin.")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data artırımı sadece train setine uygulanır
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Val ve test setlerinde sadece normalize
test_val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    'cnndata/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

val_data = test_val_datagen.flow_from_directory(
    'cnndata/valid',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_data = test_val_datagen.flow_from_directory(
    'cnndata/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Veri akışını test et
try:
    batch_images, batch_labels = next(train_data)
    print("Eğitim batch şekli:", batch_images.shape, batch_labels.shape)
    print("Eğitim etiketleri:", batch_labels)
    print("Eğitim örnek sayısı:", train_data.samples)
    print("Doğrulama örnek sayısı:", val_data.samples)
    print("Test örnek sayısı:", test_data.samples)
except Exception as e:
    raise ValueError(f"Veri akışında hata: {e}")

In [ ]:
#5 Sınıf dağılımını ve ağırlıklarını kontrol etme
# - Sınıf dengesizliğini telafi etmek için ağırlıklar hesaplanır
print("Sınıflar:", train_data.class_indices)
unique, counts = np.unique(train_data.classes, return_counts=True)
print("Sınıf dağılımı (train):", dict(zip(unique, counts)))

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))
print("Sınıf ağırlıkları:", class_weights_dict)

In [ ]:
#6 Basitleştirilmiş CNN modeli
# - İkili sınıflandırma için optimize edildi
# - Daha az katman, bellek kullanımını azaltır
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # İkili sınıflandırma için sigmoid
])

In [ ]:
#7 Modeli derleme
# - binary_crossentropy: İkili sınıflandırma için uygun
# - Adam optimizasyon, düşük öğrenme oranı
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#8 Callback’ler tanımlama
# - Erken durdurma ve en iyi modelin kaydedilmesi
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    'Web/models/best_cnn_two_class_model.h5',
    monitor='val_loss',
    save_best_only=True
)

In [ ]:
#9 Modeli eğitme
# - Küçük adımlarla eğitim, çökme riskini azaltır
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

In [ ]:
#10 Eğitim grafiklerini çizme
# - Doğruluk ve kayıp grafikleri
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Doğruluk")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Kayıp")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)

plt.tight_layout()
plt.savefig('training_plots.png')

In [ ]:
#11 Performans değerlendirme
# - model.evaluate: Genel kayıp ve doğruluk
# - classification_report: Detaylı metrikler
loss, accuracy = model.evaluate(val_data, verbose=0)
print(f"\nDoğrulama Kaybı: {loss:.4f}")
print(f"Doğrulama Doğruluğu: {accuracy:.4f}")

val_pred = model.predict(val_data)
val_pred_classes = (val_pred > 0.5).astype(int).flatten()  # Sigmoid için eşik 0.5
val_true_classes = val_data.classes
print("\nSınıflandırma Raporu:")
print(classification_report(val_true_classes, val_pred_classes, target_names=list(train_data.class_indices.keys())))

In [ ]:
#12 Modeli kaydetme
model.save('Web/models/final_cnn_two_class_model.h5')